<a href="https://colab.research.google.com/github/begzodeshmatov/suniy/blob/main/dars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

3 amaliy topshiriq. Eshmatov B

Rasm,video,audio, word va pdf fayllarini topish.

In [ ]:
pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.5 MB/s eta 0:00:00


In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.7 MB/s eta 0:00:00


Rasmni topish uchun quyidagicha kodlar yoziladi.

In [45]:
import os
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from PyPDF2 import PdfReader
from docx import Document
from fuzzywuzzy import fuzz
from PIL import Image
from io import BytesIO
import gradio as gr

# 1. Hujjatlarni va rasmlarni saqlash uchun baza
documents = []
images = []

# 2. PDF fayllarni o'qish funksiyasi
def read_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        print(f"PDF faylni o'qishda xato: {e}")
        return ""

# 3. DOCX fayllarni o'qish funksiyasi
def read_docx(file_path):
    try:
        doc = Document(file_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        return text
    except Exception as e:
        print(f"DOCX faylni o'qishda xato: {e}")
        return ""

# 4. Hujjatlarni va rasmlarni yuklash funksiyasi
def load_documents_from_folder(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_name.endswith(".pdf"):
            text = read_pdf(file_path)
            if text.strip():  # Matn bo'sh bo'lmasligi kerak
                documents.append({"text": text, "link": file_path})
        elif file_name.endswith(".docx"):
            text = read_docx(file_path)
            if text.strip():  # Matn bo'sh bo'lmasligi kerak
                documents.append({"text": text, "link": file_path})
        elif file_name.lower().endswith((".png", ".jpg", ".jpeg")):
            images.append({"name": file_name, "link": file_path})

# 5. GitHubdagi rasmlarni yuklash funksiyasi
def load_images_from_github(image_urls):
    for url in image_urls:
        try:
            response = requests.get(url)
            if response.status_code == 200:
                img_name = os.path.basename(url)
                images.append({"name": img_name, "link": url})
            else:
                print(f"Rasmni yuklashda xato: {url}")
        except Exception as e:
            print(f"GitHubdan rasm yuklashda xato: {e}")

# 6. Foydalanuvchi qidiruv tizimini yaratish
def search_documents_and_images(query):
    results = []
    image_links = []

    # 6.1 Matnli hujjatlarni qidirish
    if documents:
        texts = [doc["text"] for doc in documents]
        vectorizer = TfidfVectorizer(stop_words='english')
        doc_vectors = vectorizer.fit_transform(texts)
        query_vector = vectorizer.transform([query])

        similarities = cosine_similarity(query_vector, doc_vectors).flatten()
        sorted_indices = similarities.argsort()[::-1]

        for idx in sorted_indices:
            if similarities[idx] > 0:
                doc = documents[idx]
                results.append(f"Matn hujjati: {doc['link']} (o'xshashlik: {similarities[idx]:.2f})")

    # 6.2 Rasmlarni qidirish
    for image in images:
        similarity = fuzz.partial_ratio(query.lower(), image["name"].lower())
        if similarity >= 30:
            image_links.append({"link": image["link"], "similarity": similarity})

    return results, image_links

# 7. Natijalarni ko'rsatish funksiyasi
def display_results(query):
    # 7.1 Hujjatlarni va rasmlarni qidirish
    results, image_links = search_documents_and_images(query)

    # 7.2 Agar rasm topilmasa
    if not image_links:
        results.append("Rasm topilmadi")

    # 7.3 Rasmlarni ko'rsatish va o'xshashlik foizini chiqarmoq
    image_outputs = []
    for image_data in image_links:
        try:
            link = image_data["link"]
            similarity = image_data["similarity"]
            if link.startswith("http"):
                response = requests.get(link)
                img = Image.open(BytesIO(response.content))
            else:
                img = Image.open(link)
            image_outputs.append(img)
            results.append(f"Rasm: {link} (o'xshashlik: {similarity}%)")
        except Exception as e:
            print(f"Rasmni ochishda xato: {e}")

    # 7.4 Natijalar matnini qaytarish
    output = "Natijalar:\n\n"
    for result in results:
        output += result + "\n"

    return output, image_outputs

# 8. Gradio interfeysini yaratish
def gradio_interface(query):
    text_results, image_outputs = display_results(query)

    # Agar bir nechta rasm bo'lsa, ularni alohida qaytarish
    return text_results, image_outputs

# Fayllarni yuklash
folder_path = "./content"  # Mahalliy fayllar uchun papka yo'li
os.makedirs(folder_path, exist_ok=True)
load_documents_from_folder(folder_path)

# GitHubdan rasmlarni yuklash
github_image_urls = [
    "https://raw.githubusercontent.com/begzodeshmatov/suniy/main/begzod.jpg",
    "https://raw.githubusercontent.com/begzodeshmatov/suniy/main/elshod.jpg",
    "https://raw.githubusercontent.com/begzodeshmatov/suniy/main/samandar.jpg",
    "https://raw.githubusercontent.com/begzodeshmatov/suniy/main/turdibek.jpg",
    "https://raw.githubusercontent.com/begzodeshmatov/suniy/main/mirzabek.jpg",
]
load_images_from_github(github_image_urls)

# Gradio interfeysi
demo = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", gr.Gallery(label="Rasmlar")],
    title="Begzodning Rasmlar qidiruv Tizimi",
    description="Rasmlar bo'yicha qidiruv tizimi"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a7c45f8fc85d7bea49.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [27]:
pip install mutagen fuzzywuzzy


Qo'shiqni topish uchun quyidagicha kodlar yoziladi.

In [44]:
import os
import requests
import gradio as gr

# 1. Hujjatlar, rasmlar va qo'shiqlarni saqlash uchun baza
documents = []
images = []
songs = []

# 2. GitHub'dan MP3 qo'shiqlarni yuklash funksiyasi
def load_songs_from_github(song_urls):
    for url in song_urls:
        try:
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                song_name = os.path.basename(url)
                songs.append({"name": song_name, "link": url})
            else:
                print(f"Qo'shiqni yuklashda xato: {url} - HTTP status kodi: {response.status_code}")
        except Exception as e:
            print(f"GitHubdan qo'shiq yuklashda xato: {e}")

# 3. Foydalanuvchi qidiruv tizimini yaratish
def search_documents_images_and_songs(query):
    results = []

    # 3.1 Matnli hujjatlarni qidirish
    if documents:
        texts = [doc["text"] for doc in documents]
        # ... (qolgan kodni saqlash)

    # 3.2 Rasmlarni qidirish
    if images:
        for image in images:
            similarity = fuzz.partial_ratio(query.lower(), image["name"].lower())
            if similarity >= 30:
                results.append(f"Rasm: {image['link']} (o'xshashlik: {similarity}%)")

    # 3.3 Qo'shiqlarni qidirish
    if songs:
        for song in songs:
            similarity = fuzz.partial_ratio(query.lower(), song["name"].lower())
            if similarity >= 30:
                results.append(f"Qo'shiq: {song['link']} (o'xshashlik: {similarity}%)")

    return results

# 4. Natijalarni ko'rsatish funksiyasi
def display_results(query):
    results = search_documents_images_and_songs(query)
    output = "Natijalar:\n\n"

    if not results:
        output += "Hech qanday natija topilmadi.\n"
    else:
        for result in results:
            output += result + "\n"

    return output

# 5. Gradio interfeysini yaratish
def gradio_interface(query):
    text_results = display_results(query)

    # Qo'shiqni musiqani output sifatida Gradio interfeysida ko'rsatish
    song_output = None
    for song in songs:
        similarity = fuzz.partial_ratio(query.lower(), song["name"].lower())
        if similarity >= 30:
            song_output = song["link"]  # Audio faylni link sifatida qaytarish

    return text_results, song_output

# GitHub'dan qo'shiqlarni yuklash
github_song_urls = [
    "https://raw.githubusercontent.com/begzodeshmatov/suniymusiqaa/main/musiqa.mp3",
    "https://raw.githubusercontent.com/begzodeshmatov/suniymusiqaa/main/music.mp3",
    "https://raw.githubusercontent.com/begzodeshmatov/suniymusiqaa/main/qushiq.mp3"
]
load_songs_from_github(github_song_urls)

# Gradio interfeysi
demo = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", "audio"],  # Audio va matn natijalarini ko'rsatish
    title="Begzodning qo'shiqlar qidiruv Tizimi",
    description="Qo'shiqlar bo'yicha qidiruv tizimi"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5fa1e86fb0fc06689c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [32]:
pip install python-Levenshtein


In [33]:
pip install pymediainfo


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.5/446.5 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pymediainfo: filename=pymediainfo-6.1.0-py2.py3-none-any.whl size=9237 sha256=adda8a0e453c7918e9d8e95a6cf12e09e32a1dd487a3e4bc2b6665e430f51ac2
  Stored in directory: /root/.cache/pip/wheels/84/22/6b/374964dcdd11a5b38e46041739ca2cd5db9dc679c8373b19c3
Successfully built pymediainfo


In [34]:
pip install opencv-python

In [35]:
pip install fuzzywuzzy python-Levenshtein opencv-python


Videoni topish uchun quyidagicha kodlar yoziladi.

In [49]:
import os
import requests
import gradio as gr
from fuzzywuzzy import fuzz

# 1. Videolarni saqlash uchun baza
videos = []

# 2. GitHub'dan videolarni yuklash funksiyasi
def load_videos_from_github(video_urls):
    for url in video_urls:
        try:
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                video_name = os.path.basename(url)
                videos.append({"name": video_name, "link": url})
            else:
                print(f"Video yuklashda xato: {url} - HTTP status kodi: {response.status_code}")
        except Exception as e:
            print(f"GitHubdan video yuklashda xato: {e}")

# 3. Foydalanuvchi qidiruv tizimini yaratish
def search_videos(query):
    results = []

    # Videolarni qidirish
    if videos:
        for video in videos:
            similarity = fuzz.partial_ratio(query.lower(), video["name"].lower())
            if similarity >= 30:
                results.append(f"Video: {video['link']} (o'xshashlik: {similarity}%)")

    return results

# 4. Natijalarni ko'rsatish funksiyasi
def display_results(query):
    results = search_videos(query)
    output = "Natijalar:\n\n"

    if not results:
        output += "Hech qanday natija topilmadi.\n"
    else:
        for result in results:
            output += result + "\n"

    return output

# 5. Gradio interfeysini yaratish
def gradio_interface(query):
    text_results = display_results(query)

    # Video output sifatida Gradio interfeysida ko'rsatish
    video_output = None
    for video in videos:
        similarity = fuzz.partial_ratio(query.lower(), video["name"].lower())
        if similarity >= 30:
            video_output = video["link"]  # Video faylni link sifatida qaytarish

    return text_results, video_output

# GitHub'dan videolarni yuklash
github_video_urls = [
    "https://raw.githubusercontent.com/begzodeshmatov/suniyvideoo/main/carvideo.mp4",
    "https://raw.githubusercontent.com/begzodeshmatov/suniyvideoo/main/tabiat.mp4",
    "https://raw.githubusercontent.com/begzodeshmatov/suniyvideoo/main/osmon.mp4",
]
load_videos_from_github(github_video_urls)

# Gradio interfeysi
demo = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", "video"],  # Video va matn natijalarini ko'rsatish
    title="Begzodning Video Qidiruv Tizimi",
    description="Videolar bo'yicha qidiruv tizimi"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8f8ecfcc67334f8eea.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Pdf faylni topish uchun quyidagicha kodlar yoziladi.

In [52]:
import os
import requests
import gradio as gr
from fuzzywuzzy import fuzz

# PDF fayllarini saqlash uchun baza
pdf_files = []

# GitHub'dan PDF fayllarini yuklash funksiyasi
def load_pdfs_from_github(pdf_urls):
    for url in pdf_urls:
        try:
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                pdf_name = os.path.basename(url)
                pdf_files.append({"name": pdf_name, "link": url})
            else:
                print(f"PDF yuklashda xato: {url} - HTTP status kodi: {response.status_code}")
        except Exception as e:
            print(f"GitHubdan PDF yuklashda xato: {e}")

# Foydalanuvchi qidiruv tizimini yaratish
def search_pdfs(query):
    results = []

    # PDF fayllarini qidirish
    if pdf_files:
        for pdf in pdf_files:
            similarity = fuzz.partial_ratio(query.lower(), pdf["name"].lower())
            if similarity == 100:  # 100% o'xshashlik bo'lsa
                results.append(f"PDF: {pdf['link']} (o'xshashlik: {similarity}%)")

    return results

# Natijalarni ko'rsatish funksiyasi
def display_results(query):
    results = search_pdfs(query)
    output = "Natijalar:\n\n"

    if not results:
        output += "Hech qanday natija topilmadi.\n"
    else:
        for result in results:
            output += result + "\n"

    return output

# Gradio interfeysini yaratish
def gradio_interface(query):
    text_results = display_results(query)

    # 100% o'xshashlikka ega PDF fayllarini output sifatida ko'rsatish
    pdf_output = None
    for pdf in pdf_files:
        similarity = fuzz.partial_ratio(query.lower(), pdf["name"].lower())
        if similarity == 100:  # Faqat 100% o'xshashlikni qaytarish
            pdf_output = pdf["link"]  # PDF faylni link sifatida qaytarish

    return text_results, pdf_output

# GitHub'dan PDF fayllarini yuklash
github_pdf_urls = [
    "https://raw.githubusercontent.com/begzodeshmatov/suniypdff/main/bir.pdf",
    "https://raw.githubusercontent.com/begzodeshmatov/suniypdff/main/ikki.pdf"
]
load_pdfs_from_github(github_pdf_urls)

# Gradio interfeysi
demo = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", "file"],  # PDF fayllarini yuklab olish uchun
    title="Begzodning PDF Qidiruv Tizimi",
    description="PDF fayllar bo'yicha qidiruv tizimi"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://71b480d7bea69481e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Word faylini topish uchun quyidagicha kodlar yoziladi.

In [53]:
import os
import requests
import gradio as gr
from fuzzywuzzy import fuzz

# Word fayllarini saqlash uchun baza
word_files = []

# GitHub'dan Word fayllarini yuklash funksiyasi
def load_word_files_from_github(word_urls):
    for url in word_urls:
        try:
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                word_name = os.path.basename(url)
                word_files.append({"name": word_name, "link": url})
            else:
                print(f"Word faylini yuklashda xato: {url} - HTTP status kodi: {response.status_code}")
        except Exception as e:
            print(f"GitHubdan Word faylini yuklashda xato: {e}")

# Foydalanuvchi qidiruv tizimini yaratish
def search_word_files(query):
    results = []

    # Word fayllarini qidirish
    if word_files:
        for word in word_files:
            similarity = fuzz.partial_ratio(query.lower(), word["name"].lower())
            if similarity == 100:  # 100% o'xshashlik bo'lsa
                results.append(f"Word fayli: {word['link']} (o'xshashlik: {similarity}%)")

    return results

# Natijalarni ko'rsatish funksiyasi
def display_results(query):
    results = search_word_files(query)
    output = "Natijalar:\n\n"

    if not results:
        output += "Hech qanday natija topilmadi.\n"
    else:
        for result in results:
            output += result + "\n"

    return output

# Gradio interfeysini yaratish
def gradio_interface(query):
    text_results = display_results(query)

    # 100% o'xshashlikka ega Word fayllarini output sifatida ko'rsatish
    word_output = None
    for word in word_files:
        similarity = fuzz.partial_ratio(query.lower(), word["name"].lower())
        if similarity == 100:  # Faqat 100% o'xshashlikni qaytarish
            word_output = word["link"]  # Word faylini link sifatida qaytarish

    return text_results, word_output

# GitHub'dan Word fayllarini yuklash
github_word_urls = [
    "https://raw.githubusercontent.com/begzodeshmatov/suniywordd/main/asdf.docx",
    "https://raw.githubusercontent.com/begzodeshmatov/suniywordd/main/qwert.docx"
]
load_word_files_from_github(github_word_urls)

# Gradio interfeysi
demo = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs=["text", "file"],  # Word fayllarini yuklab olish uchun
    title="Begzodning Word Qidiruv Tizimi",
    description="Word fayllar bo'yicha qidiruv tizimi"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://00219c2f0170ed771a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
